<p style="align: center;">
    <img align=center src="../img/dls_logo.jpg" width=500 height=500>
</p>

<h1 style="text-align: center;">
    <b>Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ</b>
</h1>

---

In [3]:
import numpy as np

---

## Верхнетреугольная матрица

Напишите код, чтобы получить **верхнетреугольную матрицу** (это такая матрица, у которой все числа под главной диагональю равны 0), обнулив все ненужные элементы.

Самое простое решение - загуглить, потому что почти для всего в `NumPy` есть готовое решение!

In [4]:
def get_triangular_matrix(A):
    return np.triu(A)

In [5]:
A = np.array([
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 9]
])

get_triangular_matrix(A)

array([[1, 2, 3],
       [0, 5, 6],
       [0, 0, 9]])

---

## Broadcasting

На самом деле, можно выполнять арифметические операции над массивами с несовпадающими размерами. Это называется [**broadcasting**](https://numpy.org/doc/stable/user/basics.broadcasting.html). В процессе броадкастингa `NumPy` пытается сделать `shape` обоих массивов одинаковым, чтобы выполнить нужную операцию.

Для этого проводится сравнение размерностей, начиная с последних размерностей. Размерности совместимы, если:

1. Количество элементов по этой размерности совпадает.
2. Количество элементов в одной из размернойстей равно 1.

Допустим, у нас было два массива, которые мы складываем:

```python
shape_1 = (1, 2, 1)
shape_2 = (2, 100)
```

Тогда можно считать, что после броадкастинга получатся массивы с `shape = (1, 2, 100)` и уже они сложатся.

In [6]:
a = np.array([1, 2, 3])
b = np.array([
    [0, 1, 2],
    [3, 4, 5]
])

print(a.shape)
print(b.shape)
print((a + b).shape)

(3,)
(2, 3)
(2, 3)


In [8]:
# ещё один пример
a = a.reshape(1, 1, 3)
print(a.shape)
print(b.shape)
print((a + b).shape)

(1, 1, 3)
(2, 3)
(1, 2, 3)


Вообще говоря, вместо использование **broadcating**'а лучше в явном виде прописывать все размерности. Если размерности обоих тензоров совпадают (например, они оба трёхмерные), тогда необходимо, чтобы соответствующие элементы кортежей формы либо совпадали, либо один из них был равен 1.

Пример:

In [9]:
a + b[np.newaxis, :, :].shape

array([[[2, 4, 6]]])

---

## Ромбик

Напишите код, чтобы в квадратной матрице $5 \times 5$ из нулей получить ромб из единиц, который касается середин сторон квадрата.

In [18]:
A = np.zeros((5, 5))

row_dist = col_dist = np.abs(np.linspace(-2, 2, 5))

dist = row_dist[np.newaxis, :] + col_dist[:, np.newaxis]

B = (dist == 2).astype(int)
B

array([[0, 0, 1, 0, 0],
       [0, 1, 0, 1, 0],
       [1, 0, 0, 0, 1],
       [0, 1, 0, 1, 0],
       [0, 0, 1, 0, 0]])

---

## Как правильно выбрать оси?

**Безобидная задача на кумулятивные суммы**

Дана матрица $M \times N$. Напишите функцию, которая возвращает вектор средних значений по вертикали.

In [19]:
def vertical_means(A):
    return A.mean(axis=0)

In [23]:
A = np.random.randint(0, 10, (5, 5))
print(A)

vertical_means(A)

[[0 6 2 9 1]
 [9 7 7 9 0]
 [3 4 3 1 0]
 [1 7 8 6 1]
 [9 0 1 7 2]]


array([4.4, 4.8, 4.2, 6.4, 0.8])

**Проблемы:**

* Что такое вертикальная ось? 
* По какой оси необходимо суммировать?
* Как не ошибиться?

**Ответ:** Операции всегда производятся по той оси, которая **исчезнет** после применения операции.

---

## Батчи

Как вы, возможно, знаете, обучение нейронных сетей происходит последовательной подачей на вход нейронной сети объектов из обучающей выборки. Представим, например, что объекты - это картинки в формате RGB.

Чтобы нейронная сеть обучалась быстрее, объекты в неё подаются не по одному, а **батчами** из $N$ объектов. Итак, на вход нейронной сети подаётся четырёхмерный (!!!) тензор формы `(batch_size, num_channels, height, width)`. Наверное, вы уже убедились, что знать, где правильно ставить оси, просто необходимо.

Благо, есть простое решение: можно просто **НЕ ТРОГАТЬ** нулевую ось.

---

## Стандартизация картинки

На диске лежит файл `image_batch.npy` (здесь в ноутбуке мы просто генеируем батч с помощью функции `np.random.randint`). В нём лежит батч (массив) картинок в формате `NumPy`. Каждая картинка задана как трёхмерная матрица формата `(num_channels, width, height)`. Нужно стандартизировать каналы одного пикселя, т.е. сделать так, чтобы для каждого пикселя среднее по всем каналам стало равно $0$, а стандартное отклонение - $1$.

In [26]:
def normalize_pictures(A):
    """
    param A: np.array[batch_size, num_channels, width, height]
    """
    
    mu = A.mean(axis=(2, 3))
    sigma = A.std(axis=(2, 3))
    
    return (A - mu[:, :, np.newaxis, np.newaxis]) / sigma[:, :, np.newaxis, np.newaxis]

In [27]:
batch = np.random.randint(0, 256, (100, 3, 300, 300))

norm_batch = normalize_pictures(batch)

norm_batch.mean(axis=(2, 3)), norm_batch.std(axis=(2, 3))

(array([[-9.19758097e-17,  3.03164901e-17,  1.95793998e-17],
        [ 6.97121373e-17, -2.90927776e-17,  1.29476676e-17],
        [-7.28306304e-17, -3.91587997e-17,  1.42108547e-17],
        [ 9.16600129e-17,  6.11461499e-17, -1.49213975e-17],
        [-2.54216401e-17,  3.78956126e-18,  3.94745964e-18],
        [-6.18961672e-17, -8.13176686e-18,  3.78956126e-17],
        [ 1.70530257e-17, -1.81583144e-17, -2.36847579e-18],
        [ 4.98958899e-17,  1.59477370e-17, -6.53699317e-17],
        [-8.84230960e-18, -6.78963059e-17,  6.31593543e-18],
        [-8.03702783e-17, -4.87116520e-17,  4.72116173e-17],
        [-8.86599436e-17, -2.43163514e-17,  3.59218828e-17],
        [-4.76853125e-17, -6.71857631e-17,  9.47390314e-18],
        [-6.28435575e-17,  3.17375755e-17, -9.47390314e-17],
        [ 2.57374369e-17, -4.57905319e-18, -2.18689264e-17],
        [ 4.26325641e-17,  3.33165594e-17, -7.65017679e-17],
        [-2.13162821e-17, -3.63166287e-17,  2.52637417e-18],
        [ 8.88967912e-17

---

## Стандартизация и транспонирование 

Задание то же самое, но нужно стандартизировать все пиксели внутри каждой картинки и сменить формат `(batch_size, num_channels, x_coord, y_coord)` на `(batch_size, x_coord, y_coord, num_channels)`. Такой формат обычно удобнее для разных вычислений и вообще выглядит более естественным, но GPU более эффективно работают с первым.

In [ ]:
def normalize_and_transpose_pictures(A):
    """
    param A: np.array[batch_size, num_channels, width, height]
    """
    
    <YOUR CODE>
    
    return <YOUR CODE>

In [ ]:
res_transposed = normalize_and_transpose_pictures(batch) 
res_transposed.mean(axis=(1, 2))

---

## Сжатие фотографии

Теперь мы хотим снизить качество батча фотографий - разбить каждое изображение на квадратики $2 \times 2$ и усреднить внутри них значения по каждому каналу отдельно.

In [ ]:
def low_quality(batch_images):
    """
    <Write parameter shapes>
    """
    
    <YOUR CODE>
    
    return <YOUR CODE>

Протестируем на реальной картинке:

In [ ]:
import imageio
import matplotlib.pyplot as plt
%matplotlib inline

img = imageio.imread('./rose.jpeg')
print(img.shape)
# в картинке сначала идут оси и только потом каналы + ее стороны не четные
# Избавимся от нечетности
img_padded = np.pad(img, [(0,1), (0, 0), (0, 0)], mode='constant')
print(img_padded.shape)
# Превратим картинку в батч
batch = np.array([img_padded])

# Применим наш код
sum_of_pixels = batch[:, ::2, ::2, :] + batch[:, ::2, 1::2, :] + batch[:, 1::2, ::2, :] + batch[:, 1::2, 1::2, :]
low_res_img = (sum_of_pixels / 4).astype(np.uint8)[0]

plt.imshow(img)
plt.show()
plt.imshow(low_res_img)
plt.show()

### Ридж
Дана квадратная матрица $A$ и массив $b$ соответствующей длины. Прибавьте элементы массива $b$ к главной диагонали матрицы $A$

In [ ]:
def upgraded_plus(A, b):
    <YOUR CODE>
    
    return <YOUR CODE>